# Summarize titers across all sera and groups
This notebook contains the analysis and code used to make summary plots for the titers from all sera analyzed in this study.

#### Initialize the notebook with required packages and read in titer files and metadata

In [1]:
#Import all required packages for this analysis
import altair as alt

import neutcurve
import numpy as np
import pandas as pd
from scipy import stats
import math
from ast import literal_eval
from statsmodels.stats.multitest import multipletests
_ = alt.data_transformers.disable_max_rows()

##### First, we read in the titer files, the strain order files, and the metadata associated with the sera analyzed in this study

In [2]:
#These are our input files. May want to add a file with the conditions so that we can plot by condition and arrange strains in the order displayed on the phylogeny
DRIVE_input_titers = snakemake.input.DRIVE_input_titers
PENN_input_titers = snakemake.input.PENN_input_titers
viral_strain_plot_order = snakemake.input.viral_strain_plot_order
metadata_csv = snakemake.input.metadata_csv

#Hard coded input files
#DRIVE_input_titers = "../results/aggregated_titers/titers_DRIVE.csv"
#PENN_input_titers = "../results/aggregated_titers/titers_PennVaccineCohort.csv"
#viral_strain_plot_order = "../data/viral_libraries/H1N1library_2023-2024_allStrains.csv"
#metadata_csv = "../data/metadata/251209_SampleMetadata_bothcohorts.csv"

#Create file names for each main text figure
#Fig2 = "../results/custom_plots/Figure2.html"
#Fig3B = "../results/custom_plots/Figure3B.html"
#Fig3C = "../results/custom_plots/Figure3C.html"
#Fig4A ="../results/custom_plots/Figure4A.html"
#Fig4B ="../results/custom_plots/Figure4B.html"
#Fig5_pre = "../results/custom_plots/Figure5_pre.html"
#Fig5_post = "../results/custom_plots/Figure5_post.html"

Fig2 = snakemake.output.Fig2
Fig3B = snakemake.output.Fig3B
Fig3C = snakemake.output.Fig3C
Fig4A =snakemake.output.Fig4A
Fig4B =snakemake.output.Fig4B
Fig5_pre = snakemake.output.Fig5_pre
Fig5_post = snakemake.output.Fig5_post

#Also generate an html with pre- and post- titers for each individual in both cohorts
titers_chart_html = snakemake.output.titers_html
#titers_chart_html ="../results/aggregated_titers/titerschart_BothCohorts.html"

NameError: name 'snakemake' is not defined

In [ ]:
#Read in titer csv as a dataframe and get the order to plot the viruses
DRIVE_titers = pd.read_csv(DRIVE_input_titers)
PENN_titers = pd.read_csv(PENN_input_titers)
metadata = pd.read_csv(metadata_csv)
viruses_to_plot = pd.read_csv(viral_strain_plot_order)
assert len(DRIVE_titers) == len(DRIVE_titers.groupby(["serum", "virus"]))
assert len(PENN_titers) == len(PENN_titers.groupby(["serum", "virus"]))
viruses = viruses_to_plot.strain.tolist()

##### Add metadata to concatenated titers files and update columns for timing and group to be more descriptive

In [ ]:
#merge dataframes for Penn and DRIVE cohortsm add metadata and rename timing variable
df = [PENN_titers, DRIVE_titers]
titers = pd.concat(df)
titers['participant'] = titers['serum'].str.split('_').str[0:3].str.join('_')
titers['day'] = titers['serum'].str.split('_').str[3]
titers.loc[titers.day == 'd0', 'time'] = "pre-vaccination"
titers.loc[titers.day == 'd28', 'time'] = "post-vaccination"
titers.loc[titers.day == 'd30', 'time'] = "post-vaccination"

#add sample metadata
titers = titers.merge(metadata, on=['group','participant'])
titers['group'] = titers['group'].str.replace('PennVaccineCohort', 'Egg Vaccine Recipients')
titers['group'] = titers['group'].str.replace('DRIVE', 'Recombinant Vaccine Recipients')


In [ ]:
#remove sample that appears to have timepoints mixed up, sample was repeated to confirm titers. All analysis was performed with and without sample to confirm that exclusion of these data do not impact the results or conclusions of this study
titers = titers[~titers['serum'].str.contains('PENN23_y1964_s006')]

##### Assign age cohorts based on birth year of individuals in study

In [ ]:
#Assign age cohorts
titers.loc[titers.YOB < 1970, 'age_cohort'] = "1949-1969"
titers.loc[titers.YOB > 1969, 'age_cohort'] = "1970-1979"
titers.loc[titers.YOB > 1979, 'age_cohort'] = "1980-1989"
titers.loc[titers.YOB > 1990, 'age_cohort'] = "1990-2001"

##### Designate which strains are included in recent strain plots

In [ ]:
#Add strain category groups to make it easier to look at just recent strains or vaccine strains alone
titers['strain_category'] = "RecentStrain"
titers.loc[titers.virus.str.contains('A/Victoria/4897/2022|A/Wisconsin/588/2019|A/California/07/2009|A/Michigan/45/2015|A/Brisbane/02/2018|A/Hawaii/70/2019'), 'strain_category'] = "VaccineStrain"

#### Calculate median titers for groups based on age, vaccine history, and time (pre-/post-) that samples were collected

In [ ]:
#Group titers by age, vaccine, and timepoint and calculate medians for these groups
titers_median_age = titers.groupby(['virus','age_cohort','time','group']).median(numeric_only=True).reset_index().rename(columns={'titer':'median_titer'}).drop(columns=['titer_sem','n_replicates'])
titers_withage_andmedian = titers.merge(titers_median_age, on=['virus','age_cohort','time','group'])

titers_median_treatment = titers.groupby(['virus','previous_vaccines','time','group']).median(numeric_only=True).reset_index().rename(columns={'titer':'median_titer'}).drop(columns=['titer_sem','n_replicates'])
titers_withtreatment_andmedian = titers.merge(titers_median_treatment, on=['virus','previous_vaccines','time','group'])

titers_median_time = titers.groupby(['virus','time','group']).median(numeric_only=True).reset_index().rename(columns={'titer':'median_titer'}).drop(columns=['titer_sem','n_replicates'])
titers_median_all_merged = titers.merge(titers_median_time, on=['virus','time','group'])

#### Generate tables to examine numbers of individuals in different groups that are being analyzed (i.e. by age cohort and by vaccine history).

In [ ]:
#Here, I wanted to look at how many people were in each group, so I selected titers just for the Wisconsin/67/2022 strain, added a column to enumerate, and then grouped by groups of interest
titers_CellVaccine = titers.loc[titers['virus']=='A/Wisconsin/67/2022']
titers_CellVaccine['count'] = 1
titers_CellVaccine_pre = titers_CellVaccine.loc[titers_CellVaccine['time']=='pre-vaccination']
count_groups_age = titers_CellVaccine_pre.groupby(['group','age_cohort']).sum(numeric_only=True)
count_groups_vaccines = titers_CellVaccine_pre.groupby(['group','previous_vaccines']).sum(numeric_only=True)
count_groups_vaccines_median = titers_CellVaccine_pre.groupby(['group']).median(numeric_only=True)


def iqr(column):
    q25, q75 = column.quantile([0.25, 0.75])
    return q75 - q25

count_groups_vaccines_iqr = titers_CellVaccine_pre.groupby(['group'])['YOB'].agg(iqr)

print("Distribution of individuals in different age cohorts:")
display(count_groups_age.drop(columns=['titer','titer_sem','n_replicates','YOB']))
print("Distribution of individuals in different vaccine history groups:")
display(count_groups_vaccines.drop(columns=['titer','titer_sem','n_replicates','YOB']))
print("Distribution of individuals in different age cohorts:")
display(count_groups_vaccines_median.drop(columns=['titer','titer_sem','n_replicates','count']))
print("Average age of individuals in both cohorts with interquartile range:")
display(count_groups_vaccines_iqr)

### Generate desired plots to make comparisons on titer data:

#### First generate plot that compares pre and post vaccination titers for all individuals in both groups
This is Figure 1 in the manuscript

In [ ]:
#Create a function that generates a plot with medians and errorband

def makeerrorband_chart(dataframe,xaxis,yaxis,median,colorparam,colors,yaxis_range):
    errorband_chart = alt.Chart(dataframe).mark_errorband(opacity=0.2,extent="iqr",).encode(
        alt.Y(yaxis,scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=20,labelFontSize=20, title="titer")),
        alt.X(xaxis, sort=viruses, axis=alt.Axis(title=None, titleFontSize=20,
                                                     labelFontSize=18,labelLimit=400)),
        alt.Color(colorparam, title=colorparam,scale=alt.
                        Scale(domain=yaxis_range, range=colors)),
    ).properties(
        height=300,
        width = alt.Step(18))
    medianline_chart = alt.Chart().mark_line(point=False,strokeWidth=2).encode(
        y=alt.X(median,scale=alt.Scale(domain=plot_range, nice=False)),
        x=alt.Y(xaxis, sort=viruses,
                title=xaxis),
        color=alt.Color(colorparam,title=colorparam,scale=alt.
                        Scale(domain=yaxis_range, range=colors)),
    )
    medianline_chart_point = alt.Chart().mark_point(filled=True, size=150).encode(
        y=alt.X(median,scale=alt.Scale(domain=plot_range, nice=False)),
        x=alt.Y(xaxis, sort=viruses,
                title=xaxis),
        color=alt.Color(colorparam,title=colorparam,scale=alt.
                        Scale(domain=yaxis_range, range=colors)),
        size=alt.value(150),
        opacity=alt.value(1),
    )
    return errorband_chart,medianline_chart,medianline_chart_point


In [ ]:
#Plot median titers pre and post vaccination for both groups (Fig 2)
domain_ = ["pre-vaccination","post-vaccination"]
plot_range = [40,4000]
range_ = ['rebeccapurple', 'firebrick']
range_ = ['#55A079', '#BF3D6F']
range_ = ['#078088','#6F1663']

#removes strains from plot that were chosen to show differences in vaccine strains and not as representative circulating strains for this season
non_selectedstrains =['A/Oregon/Flu-OHSU-241140095/2023','A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Victoria/4897/2022_IVR238','A/Wisconsin/67/2022']

source = titers_median_all_merged.loc[~titers_median_all_merged['virus'].isin(non_selectedstrains)]
source = source.loc[source['group'].str.contains("Egg|Recombinant")]

errorband_chart,medianline_chart,medianline_point = makeerrorband_chart(source,"virus","titer","median_titer","time",range_,domain_)
median_chart = alt.layer(errorband_chart,medianline_chart,medianline_point, data = source).facet(row=alt.Row('group', header=alt.Header(title=None, labelFontSize=24, labelFontStyle="bold", labelPadding=0), sort=['Egg Vaccine Recipients','DRIVE'])).configure_axis(grid=False, labelLimit=300,labelFontSize=12).configure_legend(titleFontSize=32, labelFontSize=24,labelLimit=300)

display(median_chart)
median_chart.save(Fig2)

##### Also complete statistical analysis to determine if difference in titer between pre- and post-vaccination samples is significant for each group
Complete a Wilcoxon test, and include multiple testing correction

In [ ]:
#Check each virus to determine if statistically significant difference between pre and post vaccine titers in each group

#Define wilcoxon test
def test_wilcoxon_prepost(comparison,virus,pre,post):
    test = stats.wilcoxon(pre,post)
    out = [comparison,virus,test[1]]
    return(out)

#Create dataframe with pvalues for each virus, using a Wilcoxon test. Performed for egg and recombinant vaccine recipients for each virus.
source = titers_median_all_merged

#Initialize a dataframe to hold pvalues
tests_and_pvalues = []
column_names = ["comparison", "virus", "pvalue"]
for virus in viruses:
    titers_virus = source.loc[source['virus'] == virus]
#Test all recombinant recipients
    titers_totest_recombinant = titers_virus.loc[titers_virus['group'] == "Recombinant Vaccine Recipients"]
    tests_and_pvalues.append(test_wilcoxon_prepost('Recombinant pre vs post',virus,titers_totest_recombinant.query("time == 'pre-vaccination'")['titer'],titers_totest_recombinant.query("time == 'post-vaccination'")['titer']))
#Test all egg recipients
    titers_totest_egg = titers_virus.loc[titers_virus['group'] == "Egg Vaccine Recipients"]
    tests_and_pvalues.append(test_wilcoxon_prepost('Egg pre vs post',virus,titers_totest_egg.query("time == 'pre-vaccination'")['titer'],titers_totest_egg.query("time == 'post-vaccination'")['titer']))
allviruses_compareprepost_bothgroups = pd.DataFrame(tests_and_pvalues, columns=column_names)


#Add multiple testing correction
column_names = ["comparison", "virus", "pvalue_bh","reject_bh","pvalue_bonf","reject_bonf"]
#Initialize a dataframe with corrected pvalues
allviruses_compareprepost_bothgroups_correctedpvalues = []

def return_bh_and_bonf_corrected_pvalues(pvaluelist):
    #Apply Benjamini-Hochberg correction
    reject_bh, p_values_corrected_bh, _, _ = multipletests(pvaluelist, method='fdr_bh')
    #Apply Bonferroni correction (for comparison)
    reject_bonf, p_values_corrected_bonf, _, _ = multipletests(pvaluelist, method='bonferroni')
    return(reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf)

#Perform first for egg recipients
egg_vaccine_recipients_pvalueslist = allviruses_compareprepost_bothgroups.loc[allviruses_compareprepost_bothgroups['comparison'] == 'Egg pre vs post']['pvalue'].tolist()
egg_vaccine_recipients_names = allviruses_compareprepost_bothgroups.loc[allviruses_compareprepost_bothgroups['comparison'] == 'Egg pre vs post']['virus'].tolist()
reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf = return_bh_and_bonf_corrected_pvalues(egg_vaccine_recipients_pvalueslist)
for i, name in enumerate(egg_vaccine_recipients_names):
    out = ['Egg pre vs post',name,p_values_corrected_bh[i],reject_bh[i],p_values_corrected_bonf[i],reject_bonf[i]]
    allviruses_compareprepost_bothgroups_correctedpvalues.append(out)

#Perform for recombinant recipients
recombinant_vaccine_recipients_pvalueslist = allviruses_compareprepost_bothgroups.loc[allviruses_compareprepost_bothgroups['comparison'] == 'Recombinant pre vs post']['pvalue'].tolist()
recombinant_vaccine_recipients_names = allviruses_compareprepost_bothgroups.loc[allviruses_compareprepost_bothgroups['comparison'] == 'Recombinant pre vs post']['virus'].tolist()
reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf = return_bh_and_bonf_corrected_pvalues(recombinant_vaccine_recipients_pvalueslist)
for i, name in enumerate(recombinant_vaccine_recipients_names):
    out = ['Recombinant pre vs post',name,p_values_corrected_bh[i],reject_bh[i],p_values_corrected_bonf[i],reject_bonf[i]]
    allviruses_compareprepost_bothgroups_correctedpvalues.append(out)

#Create dataframe with corrected pvalues
allviruses_compareprepost_bothgroups_correctedpvalues_df = pd.DataFrame(allviruses_compareprepost_bothgroups_correctedpvalues, columns=column_names)

#Check if any comparisons are no longer significant after Bonferroni correction
print("\nDifferences that are not significant, after Bonferroni correction:")
display(allviruses_compareprepost_bothgroups_correctedpvalues_df.loc[allviruses_compareprepost_bothgroups_correctedpvalues_df['reject_bonf']==False])
print("\nAll corrected pvalues >= 0.05:")
display(allviruses_compareprepost_bothgroups_correctedpvalues_df.loc[allviruses_compareprepost_bothgroups_correctedpvalues_df['pvalue_bonf']>=0.05])
print("\nAll corrected pvalues are <= 0.05. Display head of dataframe")
allviruses_compareprepost_bothgroups_correctedpvalues_df

#### Next generate plot that compares pre and post vaccination titers and fold-changes for all vaccine strains for all individuals in both groups
This is Supplemental Figure 3 in the manuscript

Define the plot style:

In [ ]:
#Define a plot for vaccine strains, showing all titer or fold-change measurements, as well as median values and error bars for iqr
def scatter_vaccine_plot(xaxis,yaxis,median,yaxistitle,dataframe):
    jitter_points = alt.Chart(dataframe).mark_circle(size=25, opacity=0.3,color="grey").encode(
        y=alt.X(yaxis,scale=alt.Scale(type='log',domain=plot_range, nice=False),title=yaxistitle),
        x=alt.Y(xaxis, sort=titer_order,
            title=None),
        xOffset="jitter:Q",
        ).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    ).properties(
        height=150,
        width = alt.Step(30))
    median_ticks = alt.Chart().mark_tick(thickness=3, opacity=1).encode(
        y=alt.X(median,scale=alt.Scale(type='log',domain=plot_range, nice=False),title=yaxistitle),
        x=alt.Y(xaxis, sort=titer_order,
                title=None),
        color=alt.Color("group", title="vaccines",scale=alt.
                        Scale(range=range_)),
        size=alt.value(25),
    )
    error_bars = alt.Chart().mark_errorbar(extent="iqr",ticks=True,thickness=2).encode(
        y=alt.X(yaxis,scale=alt.Scale(type='log',domain=plot_range, nice=False),title=yaxistitle),
        x=alt.Y(xaxis,sort=titer_order),
        color=alt.Color("group", title="vaccines",scale=alt.Scale(range=range_)),
    )
    return(jitter_points,median_ticks,error_bars)

##### Plot the titers as points and the median as tick marks. Include error bars that show the IQR.

In [ ]:
#Plot neutralization titers for vaccine strains for both groups
domain_ = ["d0","d30"]
plot_range = [40,50000]
range_ = ['#BF3D6F','#55A079']
vaccine_strains =['A/California/07/2009',
'A/Michigan/45/2015',
'A/Brisbane/02/2018',
'A/Hawaii/70/2019',
'A/Wisconsin/588/2019','A/Wisconsin/67/2022','A/Victoria/4897/2022_IVR238']

source = titers_median_all_merged.loc[titers_median_all_merged['virus'].isin(vaccine_strains)]
source['x-axis'] = source['virus'] +"_" + source['group']

titer_order =['A/California/07/2009_Egg Vaccine Recipients','A/California/07/2009_Recombinant Vaccine Recipients',
'A/Michigan/45/2015_Egg Vaccine Recipients','A/Michigan/45/2015_Recombinant Vaccine Recipients',
'A/Brisbane/02/2018_Egg Vaccine Recipients','A/Brisbane/02/2018_Recombinant Vaccine Recipients',
'A/Hawaii/70/2019_Egg Vaccine Recipients','A/Hawaii/70/2019_Recombinant Vaccine Recipients',
'A/Wisconsin/588/2019_Egg Vaccine Recipients','A/Wisconsin/588/2019_Recombinant Vaccine Recipients',
'A/Wisconsin/67/2022_Egg Vaccine Recipients','A/Wisconsin/67/2022_Recombinant Vaccine Recipients',
'A/Victoria/4897/2022_IVR238_Egg Vaccine Recipients','A/Victoria/4897/2022_IVR238_Recombinant Vaccine Recipients']

jitter_points, median_ticks, error_bars = scatter_vaccine_plot('x-axis','titer','median_titer','neutralization titer',source)
median_chart = alt.layer(jitter_points,median_ticks,error_bars, data=source).facet(row=alt.Row('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)

display(median_chart)

##### Perform testing to determine if any titers are significantly different between groups.
All vaccine strains are different except A/California/07/2009 and A/Victoria.4897/2022, but this may just be due to differences in initial titers between groups

In [ ]:
#Test significance of difference between vaccine groups
source = titers_median_all_merged.loc[titers_median_all_merged['virus'].isin(vaccine_strains)]
print("Vaccine strains that are significantly different between egg and recombinant vaccines pre-vaccination:")
for virus in vaccine_strains:
    titers_totest = source.loc[source['virus'] == virus]
    titers_totest = titers_totest.loc[titers_totest['time'] == "pre-vaccination"]
    test = stats.mannwhitneyu(titers_totest.query("group == 'Egg Vaccine Recipients'")['titer'], titers_totest.query("group == 'Recombinant Vaccine Recipients'")['titer'])
    if test[1] < 0.05:
        print(virus, test[1])

print("\nVaccine strains that are significantly different between egg and recombinant vaccines post-vaccination:")
source = titers_median_all_merged.loc[titers_median_all_merged['virus'].isin(vaccine_strains)]
for virus in vaccine_strains:
    titers_totest = source.loc[source['virus'] == virus]
    titers_totest = titers_totest.loc[titers_totest['time'] == "post-vaccination"]
    test = stats.mannwhitneyu(titers_totest.query("group == 'Egg Vaccine Recipients'")['titer'], titers_totest.query("group == 'Recombinant Vaccine Recipients'")['titer'])
    if test[1] < 0.05:
        print(virus, test[1])

##### Calculate fold-change for titer for each individual and then use same plot structure to visualize these data

In [ ]:
#Pivot the vaccine titers dataframe to calculate fold-changes
titers_vaccines_pivot = pd.pivot_table(titers_median_all_merged, values='titer', index=['group','participant','virus'],columns=['time']).reset_index()
#Calculate fold-change
titers_vaccines_pivot['fold-change'] = titers_vaccines_pivot['post-vaccination'] / titers_vaccines_pivot['pre-vaccination']
#Calculate median fold-change for each group
titers__vaccines_pivot_median = titers_vaccines_pivot.groupby(['virus','group']).median(numeric_only=True).reset_index().rename(columns={'fold-change':'median_foldchange'})
#Add median fold-change to dataframe
titers_pivot_vaccines_median_merged = titers_vaccines_pivot.merge(titers__vaccines_pivot_median, on=['virus','group'])


##### Perform testing to determine if any fold-changes are significantly different between groups.
The only vaccine strain that is different is the A/Wisconsin/67/2022 strain.

In [ ]:
#Check if any fold-changes are statistically significant
source = titers_pivot_vaccines_median_merged.loc[titers_pivot_vaccines_median_merged['virus'].isin(vaccine_strains)]

print("\nVaccine strains with significantly different fold-changes between egg and recombinant vaccinees:")
for virus in vaccine_strains:
    titers_totest = source.loc[source['virus'] == virus]
    test = stats.mannwhitneyu(titers_totest.query("group == 'Egg Vaccine Recipients'")['fold-change'], titers_totest.query("group == 'Recombinant Vaccine Recipients'")['fold-change'])
    if test[1] < 0.05:
        print(virus, test[1])

In [ ]:
#Plot fold-changes in vaccine strains
domain_ = ["d0","d30"]
plot_range = [0.1,1000]
range_ = ['#BF3D6F','#55A079']
vaccine_strains =['A/California/07/2009',
'A/Michigan/45/2015',
'A/Brisbane/02/2018',
'A/Hawaii/70/2019',
'A/Wisconsin/588/2019','A/Wisconsin/67/2022','A/Victoria/4897/2022_IVR238']

titer_order =['A/California/07/2009_Egg Vaccine Recipients','A/California/07/2009_Recombinant Vaccine Recipients',
'A/Michigan/45/2015_Egg Vaccine Recipients','A/Michigan/45/2015_Recombinant Vaccine Recipients',
'A/Brisbane/02/2018_Egg Vaccine Recipients','A/Brisbane/02/2018_Recombinant Vaccine Recipients',
'A/Hawaii/70/2019_Egg Vaccine Recipients','A/Hawaii/70/2019_Recombinant Vaccine Recipients',
'A/Wisconsin/588/2019_Egg Vaccine Recipients','A/Wisconsin/588/2019_Recombinant Vaccine Recipients',
'A/Wisconsin/67/2022_Egg Vaccine Recipients','A/Wisconsin/67/2022_Recombinant Vaccine Recipients',
'A/Victoria/4897/2022_IVR238_Egg Vaccine Recipients','A/Victoria/4897/2022_IVR238_Recombinant Vaccine Recipients']

source = titers_pivot_vaccines_median_merged.loc[titers_pivot_vaccines_median_merged['virus'].isin(vaccine_strains)]
source['x-axis'] = source['virus'] +"_" + source['group']

jitter_points, median_ticks, error_bars = scatter_vaccine_plot('x-axis','fold-change','median_foldchange','fold-change',source)
median_chart = alt.layer(jitter_points, median_ticks, error_bars, data=source).configure_axis(grid=False, labelLimit=300,labelFontSize=12)

display(median_chart)


### Next, we are going to look at fold-changes in titers to all recent strains between groups
These plots are shown in Figure 3.

##### Calculate fold change in titer for every individual

In [ ]:
#Calculate all fold changes for all viruses for all individuals
titers_pivot = pd.pivot_table(titers, values='titer', index=['group','participant','virus'],columns=['time']).reset_index()
titers_pivot['fold-change'] = titers_pivot['post-vaccination'] / titers_pivot['pre-vaccination']

titers_pivot_median = titers_pivot.groupby(['virus','group']).median(numeric_only=True).reset_index().rename(columns={'fold-change':'median_foldchange'})
titers_pivot_median_merged = titers_pivot.merge(titers_pivot_median, on=['virus','group'])

##### Next plot the data for all individuals in both groups

In [ ]:
#Plot fold-change data for all recent circulating strains
plot_range = [1,12]
range_ = ['#55A079','#BF3D6F']
domain_ = ['Egg Vaccine Recipients','Recombinant Vaccine Recipients']
source = titers_pivot_median_merged

#Remove strains included for vaccine comparisons, so just the set of strains selected for representing diversity is shown
non_recentstrainset =['A/Oregon/Flu-OHSU-241140095/2023','A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Victoria/4897/2022_IVR238','A/Wisconsin/67/2022']
source = source.loc[~source['virus'].isin(non_recentstrainset)]

#Generate errorband chart with fold-change
errorband_chart,medianline_chart,medianline_chart_point = makeerrorband_chart(source,"virus","fold-change","median_foldchange","group",range_,domain_)
median_chart = alt.layer(errorband_chart,medianline_chart_point,medianline_chart, data = source)#.facet(row=alt.Row('age_cohort', header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False)
display(median_chart)
median_chart.save(Fig3C)

##### Perform statistical analysis to look at whether this difference in fold change is significant
First define the strain groups:

In [ ]:
#Define recent strains with and without K142R and in and outside the 5a2a1 clade

R142_strains = ['A/PaisVasco/1994/2023','A/Arizona/43/2023','A/Michigan/MISAPPHIREL365374393/2023','A/Netherlands/1739/2023','A/California/82/2023','A/Gwangju/1254/2023','A/NewHampshire/7/2023','A/Norway/9845/2023','A/Denmark/1567/2023','A/Sydney/845/2023','A/Pennsylvania/46/2023','A/RioGrandedoSul/6798/2023','A/CostaRica/INC-0036-793184/2023','A/EspiritoSanto/6964/2023','A/SouthAfrica/R06646/2023','A/Sydney/130/2023','A/ChiangRai/P3437/2023','A/Victoria/1389/2023']
K142_strains = ['A/CostaRica/INC-0053-796776/2023', 'A/Victoria/1010/2023', 'A/Ghana/1127/2023', 'A/Norway/8030/2023', 'A/Washington/32/2023', 'A/Norway/7787/2023', 'A/Chungbuk/1553/2023', 'A/Victoria/1834/2023', 'A/Sydney/591/2023', 'A/Oregon/32/2023', 'A/Philippines/10/2023', 'A/Senegal/1382/2023', 'A/Jeonbuk/1544/2023', 'A/Pennsylvania/51/2023', 'A/Cambodia/KPH230176/2023', 'A/Brazil/GIHSN-HCL023128467501/2023', 'A/Para/2023-020942-IEC/2023', 'A/California/75/2023', 'A/Arizona/IVYG43Q65T3/2023', 'A/Senegal/1341/2023', 'A/Argentina/824/2023', 'A/Valparaiso/41964/2023', 'A/Panama/M270337/2023', 'A/Argentina/677/2023', 'A/Thies/1247/2023', 'A/Senegal/1423/2023', 'A/CostaRica/INC-0048-796565/2023', 'A/CostaRica/INC-0348-802602/2023', 'A/Victoria/1013/2023', 'A/NakhonPhanom/P3460/2023', 'A/Cambodia/SVH230198/2023', 'A/Sydney/166/2023', 'A/Victoria/1384/2023', 'A/Tasmania/140/2023', 'A/SouthAustralia/179/2023', 'A/Perth/110/2023', 'A/Perth/372/2023', 'A/Perth/275/2023', 'A/Victoria/2241/2023', 'A/Singapore/GP7085/2023']
clade_5a2a1 = ['A/PaisVasco/1994/2023','A/Arizona/43/2023','A/Michigan/MISAPPHIREL365374393/2023','A/Netherlands/1739/2023','A/California/82/2023','A/Gwangju/1254/2023','A/NewHampshire/7/2023','A/Norway/9845/2023','A/Denmark/1567/2023','A/Sydney/845/2023','A/Pennsylvania/46/2023','A/RioGrandedoSul/6798/2023','A/CostaRica/INC-0036-793184/2023','A/EspiritoSanto/6964/2023','A/SouthAfrica/R06646/2023','A/Sydney/130/2023']
non_5a2a1 = ['A/CostaRica/INC-0053-796776/2023', 'A/Victoria/1010/2023', 'A/Ghana/1127/2023', 'A/Norway/8030/2023', 'A/Washington/32/2023', 'A/Norway/7787/2023', 'A/Chungbuk/1553/2023', 'A/Victoria/1834/2023', 'A/Sydney/591/2023', 'A/Oregon/32/2023', 'A/Philippines/10/2023', 'A/Senegal/1382/2023', 'A/Jeonbuk/1544/2023', 'A/Pennsylvania/51/2023', 'A/Cambodia/KPH230176/2023', 'A/Brazil/GIHSN-HCL023128467501/2023', 'A/Para/2023-020942-IEC/2023', 'A/California/75/2023', 'A/Arizona/IVYG43Q65T3/2023', 'A/Senegal/1341/2023', 'A/Argentina/824/2023', 'A/Valparaiso/41964/2023', 'A/Panama/M270337/2023', 'A/Argentina/677/2023', 'A/Thies/1247/2023', 'A/Senegal/1423/2023', 'A/CostaRica/INC-0048-796565/2023', 'A/CostaRica/INC-0348-802602/2023', 'A/Victoria/1013/2023', 'A/NakhonPhanom/P3460/2023', 'A/Cambodia/SVH230198/2023', 'A/Sydney/166/2023', 'A/Victoria/1384/2023', 'A/Tasmania/140/2023', 'A/SouthAustralia/179/2023', 'A/Perth/110/2023', 'A/Perth/372/2023', 'A/Perth/275/2023','A/ChiangRai/P3437/2023','A/Victoria/1389/2023', 'A/Victoria/2241/2023', 'A/Singapore/GP7085/2023']


In [ ]:
#Check each virus to determine if statistically significant difference between pre and post vaccine titers in each group

#Define mannwhitneyu test
def test_mannwhitneyu_eggvsrecombinant(comparison,virus,egg,recombinant):
    test = stats.mannwhitneyu(egg,recombinant,alternative="less")
    out = [virus,test[1]]
    return(out)

#Create dataframe with pvalues for each virus, using a Wilcoxon test. Performed for egg and recombinant vaccine recipients for each virus.
source = titers_pivot_median_merged

#Initialize a dataframe to hold pvalues
tests_and_pvalues = []
column_names = ["virus", "pvalue"]
for virus in viruses:
    foldchange_virus = source.loc[source['virus'] == virus]
    tests_and_pvalues.append(test_mannwhitneyu_eggvsrecombinant('Fold-changes, egg vs recombinant',virus,foldchange_virus.query("group == 'Egg Vaccine Recipients'")['fold-change'], foldchange_virus.query("group == 'Recombinant Vaccine Recipients'")['fold-change']))
allviruses_comparefoldchange = pd.DataFrame(tests_and_pvalues, columns=column_names)


#Add multiple testing correction
column_names = ["comparison", "virus", "pvalue","pvalue_bh","reject_bh","pvalue_bonf","reject_bonf"]
#Initialize a dataframe with corrected pvalues
allviruses_comparefoldchange_correctedpvalues = []

def return_bh_and_bonf_corrected_pvalues(pvaluelist):
    #Apply Benjamini-Hochberg correction
    reject_bh, p_values_corrected_bh, _, _ = multipletests(pvaluelist, method='fdr_bh')
    #Apply Bonferroni correction (for comparison)
    reject_bonf, p_values_corrected_bonf, _, _ = multipletests(pvaluelist, method='bonferroni')
    return(reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf)

#Perform correction for all viruses
viruses_pvalueslist = allviruses_comparefoldchange['pvalue'].tolist()
viruses_names = allviruses_comparefoldchange['virus'].tolist()
reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf = return_bh_and_bonf_corrected_pvalues(viruses_pvalueslist)
for i, name in enumerate(viruses_names):
    out = ['Egg vs Recombinant foldchange',name,viruses_pvalueslist[i],p_values_corrected_bh[i],reject_bh[i],p_values_corrected_bonf[i],reject_bonf[i]]
    allviruses_comparefoldchange_correctedpvalues.append(out)

#Create dataframe with corrected pvalues
allviruses_comparefoldchange_correctedpvalues_df = pd.DataFrame(allviruses_comparefoldchange_correctedpvalues, columns=column_names)

#Check if any comparisons are significant after Benjamini-Hochberg correction
if len(allviruses_comparefoldchange_correctedpvalues_df.loc[allviruses_comparefoldchange_correctedpvalues_df['reject_bh']==True]) > 0:
    print("\nDifferences are significant, even after Benjamini-Hochberg correction:")
    display(allviruses_comparefoldchange_correctedpvalues_df.loc[allviruses_comparefoldchange_correctedpvalues_df['reject_bh']==True])

#Generate a list of significant viruses to compare to clade 5a2a1 and K142R strain lists
viruses_significant = allviruses_comparefoldchange_correctedpvalues_df.loc[allviruses_comparefoldchange_correctedpvalues_df['pvalue_bh']<=0.05]['virus'].tolist()

#Check for overlap with clade and strain lists
overlap_clade5a2a1 = all(item in viruses_significant for item in clade_5a2a1)
if overlap_clade5a2a1 == True:
    print("All strains in clade 5a2a1 have significantly different fold-changes between groups after Benjamini-Hochberg correction, but not Bonferroni")
else:
    print("Not strains in clade 5a2a1 have significantly different fold-changes between groups after Benjamini-Hochberg correction, but not Bonferroni")

overlap_R142 = all(item in viruses_significant for item in R142_strains)
if overlap_R142 == True:
    print("All strains with K142R have significantly different fold-changes between groups after Benjamini-Hochberg correction, but not Bonferroni")
else:
    print("Not strains with K142R have significantly different fold-changes between groups after Benjamini-Hochberg correction, but not Bonferroni")

overlap_K142 = any(item in K142_strains for item in viruses_significant)
if overlap_K142 == True:
    print("Some strains with K142 have significantly different fold-changes between groups after Benjamini-Hochberg correction")
else:
    print("No strains with K142 have significantly different fold-changes between groups after Benjamini-Hochberg correction")

##### Repeat this plot, excluding individuals that have titers below limit of detection to confirm that this is not impacting result

In [ ]:
#First define a dataframe that lacks all individuals that contain an LOD titer
titers_LOD = titers.loc[titers['titer_bound'].str.contains('upper|lower')]
participants_withLODtiter = titers_LOD['participant'].unique().tolist()
titers_noLOD = titers.loc[~titers['participant'].isin(participants_withLODtiter)]

#Then calculate fold-change for these individuals
titers_pivot_nolod = pd.pivot_table(titers_noLOD, values='titer', index=['group','participant','virus'],columns=['time']).reset_index()
titers_pivot_nolod['fold-change'] = titers_pivot_nolod['post-vaccination'] / titers_pivot_nolod['pre-vaccination']

titers_pivot_nolod_median = titers_pivot_nolod.groupby(['virus','group']).median(numeric_only=True).reset_index().rename(columns={'fold-change':'median_foldchange'})
titers_pivot_nolod_median_merged = titers_pivot_nolod.merge(titers_pivot_nolod_median, on=['virus','group'])

titers_pivot_nolod['group'].unique().tolist()
print("Number of individuals in egg vaccine group where all titers are not at LOD:",str(len(titers_pivot_nolod.loc[titers_pivot_nolod['group'].str.contains("Egg")]['participant'].unique().tolist())))

print("Number of individuals in recombinant vaccine group where all titers are not at LOD:",str(len(titers_pivot_nolod.loc[titers_pivot_nolod['group'].str.contains("Recombinant")]['participant'].unique().tolist())))

In [ ]:
#Re-plot fold-change data for all recent circulating strains for just individuals who do not have any titers at LOD
plot_range = [1,12]
range_ = ['#55A079','#BF3D6F']
domain_ = ['Egg Vaccine Recipients','Recombinant Vaccine Recipients']

source = titers_pivot_nolod_median_merged
source = source.loc[~source['virus'].isin(non_recentstrainset)]

#Generate errorband chart with fold-change
errorband_chart,medianline_chart,medianline_point = makeerrorband_chart(source,"virus","fold-change","median_foldchange","group",range_,domain_)
median_chart = alt.layer(errorband_chart,medianline_chart_point,medianline_chart, data = source)#.facet(row=alt.Row('age_cohort', header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False)
display(median_chart)

In [ ]:
#We also want to confirm that vaccine history is not biasing this result. Therefore here, I define a dataframe that contains only 3x vaccinated individuals
titers_3xVax = titers.loc[titers['previous_vaccines'].str.contains('2 Vaccines')]
participants_with3xvaccines = titers_3xVax['participant'].unique().tolist()
titers_3xVaxonly = titers.loc[titers['participant'].isin(participants_with3xvaccines)]

#Then calculate fold-change for these individuals
titers_pivot_3xvax = pd.pivot_table(titers_3xVaxonly, values='titer', index=['group','participant','virus'],columns=['time']).reset_index()
titers_pivot_3xvax['fold-change'] = titers_pivot_3xvax['post-vaccination'] / titers_pivot_3xvax['pre-vaccination']

titers_3xVaxonly_median = titers_pivot_3xvax.groupby(['virus','group']).median(numeric_only=True).reset_index().rename(columns={'fold-change':'median_foldchange'})
titers_3xVaxonly_median_merged = titers_pivot_3xvax.merge(titers_3xVaxonly_median, on=['virus','group'])

titers_pivot_3xvax['group'].unique().tolist()
print("Number of individuals in egg vaccine group with 3x vaccines:",str(len(titers_pivot_3xvax.loc[titers_pivot_3xvax['group'].str.contains("Egg")]['participant'].unique().tolist())))

print("Number of individuals in recombinant vaccine group with 3x vaccines:",str(len(titers_pivot_3xvax.loc[titers_pivot_3xvax['group'].str.contains("Recombinant")]['participant'].unique().tolist())))

In [ ]:
#Re-plot fold-change data for all recent circulating strains for just individuals with 3x vaccines
plot_range = [1,12]
range_ = ['#55A079','#BF3D6F']
domain_ = ['Egg Vaccine Recipients','Recombinant Vaccine Recipients']

source = titers_3xVaxonly_median_merged
source = source.loc[~source['virus'].isin(non_recentstrainset)]

#Generate errorband chart with fold-change
errorband_chart,medianline_chart,medianline_point = makeerrorband_chart(source,"virus","fold-change","median_foldchange","group",range_,domain_)
median_chart = alt.layer(errorband_chart,medianline_chart_point,medianline_chart, data = source)#.facet(row=alt.Row('age_cohort', header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False)
display(median_chart)

##### Next we are going to look a the fold-change in strains that represent the individual mutations that separate the egg- and recombinant vaccines in order to determine which mutations are responsible for this difference

In [ ]:
#Plot fold-changes for vaccine strains and strains that represent the single amino acid steps between vaccine
titers_pivot_median_merged

domain_ = ["pre-vaccination","post-vaccination"]
plot_range = [1,12]
range_ = ['rebeccapurple', 'firebrick']
range_ = ['#BF3D6F','#55A079']

listofstrains =['A/Wisconsin/67/2022','A/Michigan/MISAPPHIREL365374393/2023','A/Oregon/Flu-OHSU-241140095/2023','A/Victoria/4897/2022_IVR238']

source = titers_pivot_median_merged.loc[titers_pivot_median_merged['virus'].isin(listofstrains)]
source = source.loc[source['group'].str.contains('Recombinant|Egg')]

errorband_chart = alt.Chart().mark_errorband(opacity=0.2,extent="iqr",).encode(
    alt.Y("fold-change",scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=14, labelFontSize=14, title="fold-change")),
    alt.X("virus", sort=listofstrains, axis=alt.Axis(title=None,
                                                 labelFontSize=14,labelLimit=300)),
    alt.Color("group", title="Group",scale=alt.
                    Scale(range=range_)),
).properties(
    height=300,
    width = 300)
meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=3).encode(
    y=alt.X("median_foldchange",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=listofstrains,
            title="virus"),
    color=alt.Color("group",title="Group",scale=alt.
                    Scale( range=range_)),
)
meanline_chart_point = alt.Chart().mark_point(filled=True).encode(
    y=alt.X("median_foldchange",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=listofstrains,
            title="virus"),
    color=alt.Color("group",title="Group",scale=alt.
                    Scale( range=range_)),
    size=alt.value(75),
    opacity=alt.value(1),
)

median_chart = alt.layer(errorband_chart,meanline_chart,meanline_chart_point, data = source)#.facet(column=alt.Column('participant', header=alt.Header(title=None, labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=400,labelFontSize=16)

display(median_chart)
median_chart.save(Fig3B)

In [ ]:
#Check if any of these strains are significantly different between groups post-correction 
listofstrains =['A/Wisconsin/67/2022','A/Michigan/MISAPPHIREL365374393/2023','A/Oregon/Flu-OHSU-241140095/2023','A/Victoria/4897/2022_IVR238']
print("Testing for significant strains that differ only by single amino acid mutations")

sig_virusesinset = []
for virus in listofstrains:
    if virus in viruses_significant:
        sig_virusesinset.append(virus)
print("Strains that have statistically significant fold-changes between groups:",str(len(sig_virusesinset)))
allviruses_comparefoldchange_correctedpvalues_df.loc[allviruses_comparefoldchange_correctedpvalues_df['virus'].isin(sig_virusesinset)]

### Next, we are going to break down the groups based on different demographic factors (i.e. vaccine history and age) to determine if any of these factors are linked with lower titers to the 5a2a1 clade strains, compared to other recent strains, post vaccination
These plots are shown in Figure 4 and Supplemental Figure 4.

##### First, define the plot that will be used to show these data

In [ ]:
#Define a plot that will show lines for all individuals, and can be used to show medians for defined groups
def linechart_withboldedmedian(dataframe,xaxis,yaxis,yaxis_median,yaxis_title,detail1,detail2,color_param,sort_color_param,color_grouptitle):
    linecharts_all = alt.Chart(dataframe).mark_line(point=False,strokeWidth=1.5, opacity=0.5).encode(
        y=alt.X(yaxis,scale=alt.Scale(type='log',domain=plot_range, nice=False),title=yaxis_title),
        x=alt.Y(xaxis, sort=viruses,
                title=None),
        color=alt.Color(color_param,sort = vaxorder, scale=alt.
                        Scale(range=range_)),
        detail= detail1,
    ).properties(
    height=200,
    width = alt.Step(10))
    median_line = alt.Chart().mark_line(point=False,strokeWidth=2,stroke='black', opacity=0.5).encode(
        y=alt.X(yaxis_median,scale=alt.Scale(domain=plot_range, nice=False)),
        x=alt.Y(xaxis, sort=viruses,
                title=None),
        color=alt.Color(color_param,sort = sort_color_param, title=color_grouptitle,scale=alt.
                        Scale( range=range_)),
        detail=detail2,
    )
    median_points = alt.Chart().mark_circle(size=60,stroke='black', strokeWidth=1, opacity=1).encode(
        y=alt.X(yaxis_median,scale=alt.Scale(type='log',domain=plot_range, nice=False)),
        x=alt.Y(xaxis, sort=viruses,
                title=None),
        color=alt.Color(color_param,sort = sort_color_param, title=color_grouptitle,scale=alt.
                        Scale(range=range_)),
        size=alt.value(75),
    )
    return(linecharts_all,median_line,median_points)

##### Next plot the titers based on vaccine history for the individuals that recieved the recombinant vaccine

In [ ]:
#Plot titers for individuals and median per group based on vaccine history and cohort (recombinant-vaccine)

#Define colors and domain parameters and order of legend
domain_ = ["d0","d30"]
plot_range = [40,20000]
range_ = ['#034867', '#70BDD2', '#F0746E']
vaxorder = ["0 Vaccines (2021-2023)","1 Vaccine (2021-2022)","1 Vaccine (2022-2023)","2 Vaccines (2021-2022,2022-2023)"]

#Remove strains included for vaccine comparisons, so just the set of strains selected for representing diversity is shown
non_recentstrainset =['A/Oregon/Flu-OHSU-241140095/2023','A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Victoria/4897/2022_IVR238','A/Wisconsin/67/2022']
source = titers_withtreatment_andmedian.loc[~titers_withtreatment_andmedian['virus'].isin(non_recentstrainset)]

#Select recombinant vaccine recipients only
source = source.loc[source['group']=="Recombinant Vaccine Recipients"]

#Show plot for all previous vaccine groups
linecharts_all,median_line,median_points = linechart_withboldedmedian(source,'virus','titer','median_titer','neutralization titer','participant','time','previous_vaccines',vaxorder,'vaccines')
median_chart = alt.layer(linecharts_all,median_line,median_points, data=source).facet(column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
display(median_chart)

#Show plot for previous vaccine groups, splitting charts to get more visualization of the data
median_chart_split = alt.layer(linecharts_all,median_line,median_points, data=source).facet(row=alt.Row('previous_vaccines'), column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
display(median_chart_split)



##### Next plot the titers based on vaccine history for the individuals that recieved the egg vaccine

In [ ]:
#Plot titers for individuals and median per group based on vaccine history and cohort (egg-vaccine)

#Define colors and domain parameters and order of legend
domain_ = ["d0","d30"]
plot_range = [40,20000]
range_ = ['#034867','#8A6B98','#70BDD2', '#F0746E']
vaxorder = ["0 Vaccines (2021-2023)","1 Vaccine (2021-2022)","1 Vaccine (2022-2023)","2 Vaccines (2021-2022,2022-2023)"]

#Remove strains included for vaccine comparisons, so just the set of strains selected for representing diversity is shown
non_recentstrainset =['A/Oregon/Flu-OHSU-241140095/2023','A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Victoria/4897/2022_IVR238','A/Wisconsin/67/2022']
source = titers_withtreatment_andmedian.loc[~titers_withtreatment_andmedian['virus'].isin(non_recentstrainset)]

#Select recombinant vaccine recipients only
source = source.loc[source['group']=="Egg Vaccine Recipients"]

#Show plot for all previous vaccine groups
linecharts_all,median_line,median_points = linechart_withboldedmedian(source,'virus','titer','median_titer','neutralization titer','participant','time','previous_vaccines',vaxorder,'vaccines')
median_chart = alt.layer(linecharts_all,median_line,median_points, data=source).facet(column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
display(median_chart)

#Show plot for previous vaccine groups, splitting charts to get more visualization of the data
median_chart_split = alt.layer(linecharts_all,median_line,median_points, data=source).facet(row=alt.Row('previous_vaccines'), column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
display(median_chart_split)


##### Next plot the titers based on age cohort for the individuals that recieved the recombinant vaccine

In [ ]:
#Plot titers for individuals and median per group based on age cohort and vaccine recieved (recombinant-vaccine)

#Define colors and domain parameters and order of legend
domain_ = ["d0","d30"]
plot_range = [40,20000]
range_ = ['#8A6B98','#70BDD2', '#F0746E']
age_cohortorder = ["1949-1969","1970-1979","1980-1989","1990-2001"]

#Remove strains included for vaccine comparisons, so just the set of strains selected for representing diversity is shown
non_recentstrainset =['A/Oregon/Flu-OHSU-241140095/2023','A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Victoria/4897/2022_IVR238','A/Wisconsin/67/2022']
source = titers_withage_andmedian.loc[~titers_withage_andmedian['virus'].isin(non_recentstrainset)]

#Select recombinant vaccine recipients only
source = source.loc[source['group']=="Recombinant Vaccine Recipients"]

#Show plot for all previous vaccine groups
linecharts_all,median_line,median_points = linechart_withboldedmedian(source,'virus','titer','median_titer','neutralization titer','participant','time','age_cohort',age_cohortorder,'Age cohort')
median_chart = alt.layer(linecharts_all,median_line,median_points, data=source).facet(column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
display(median_chart)

#Show plot for previous vaccine groups, splitting charts to get more visualization of the data
median_chart_split = alt.layer(linecharts_all,median_line,median_points, data=source).facet(row=alt.Row('age_cohort'), column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
#display(median_chart_split)
median_chart.save(Fig4B)

##### Finally, plot the titers based on age cohort for the individuals that recieved the egg vaccine

In [ ]:
#Plot titers for individuals and median per group based on age cohort and vaccine recieved (recombinant-vaccine)

#Define colors and domain parameters and order of legend
domain_ = ["d0","d30"]
plot_range = [40,20000]
range_ = ['#034867','#8A6B98','#70BDD2', '#F0746E']
age_cohortorder = ["1949-1969","1970-1979","1980-1989","1990-2001"]

#Remove strains included for vaccine comparisons, so just the set of strains selected for representing diversity is shown
non_recentstrainset =['A/Oregon/Flu-OHSU-241140095/2023','A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Victoria/4897/2022_IVR238','A/Wisconsin/67/2022']
source = titers_withage_andmedian.loc[~titers_withage_andmedian['virus'].isin(non_recentstrainset)]

#Select recombinant vaccine recipients only
source = source.loc[source['group']=="Egg Vaccine Recipients"]

#Show plot for all previous vaccine groups
linecharts_all,median_line,median_points = linechart_withboldedmedian(source,'virus','titer','median_titer','neutralization titer','participant','time','age_cohort',age_cohortorder,'Age cohort')
median_chart = alt.layer(linecharts_all,median_line,median_points, data=source).facet(column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
display(median_chart)

#Show plot for previous vaccine groups, splitting charts to get more visualization of the data
median_chart_split = alt.layer(linecharts_all,median_line,median_points, data=source).facet(row=alt.Row('age_cohort'), column=alt.Column('time', sort=['pre-vaccination','post-vaccination'],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False, labelLimit=300,labelFontSize=12)
#display(median_chart_split)
median_chart.save(Fig4A)

#### Create a chart that shows the pre- and post-vaccination titer profiles for each individual

In [ ]:
#Define code to generate titer plot
def run_titerchart_rot(dataframe, ncols=10):

    virus_selection = alt.selection_point(fields=["virus"], on="mouseover", empty=False)

    serum_selection = alt.selection_point(
        fields=["participant"],
        bind="legend",
        toggle="true",
    )

    sera = dataframe["participant"].unique().tolist()

    titers_chart = (
        alt.Chart(dataframe)
        .add_params(virus_selection, serum_selection)
        .transform_filter(serum_selection)
        .encode(
            alt.Y(
                "titer",
                title="neutralization titer",
                scale=alt.Scale(nice=False, padding=4, type="log"),
                axis=alt.Axis(labelOverlap=True,labelFontSize=15,titleFontSize=16),
            ),
            alt.X("virus", sort=viruses,axis=alt.Axis(title=None,labelFontSize=12,labelLimit=300)),
            alt.Color("time", sort=condition_order, title="Timepoint",scale=alt.Scale(range=['#078088', '#6F1663'])),
            alt.Shape("titer_bound"),
            alt.Facet(
                "participant:N",
                header=alt.Header(
                    title=None, labelFontSize=14, labelFontStyle="bold", labelPadding=0
                ),
                spacing=3,
                columns=ncols,
            ),
            alt.StrokeWidth(
                "participant:N",
                scale=alt.Scale(domain=sera, range=[1] * len(sera)),
                legend=alt.Legend(
                    orient="bottom",
                    columns=ncols,
                    symbolLimit=0,
                    symbolFillColor="black",
                    title="participant (click to select)",
                ),
            ),
        )
        .mark_line(point=True)
        .configure_axis(grid=False,labelLimit=300)
        .configure_point(size=45)
        .configure_legend(titleColor='black', titleFontSize=16, labelFontSize=16)
        .properties(
            width=alt.Step(11),
            height=100,
            autosize=alt.AutoSizeParams(resize=True),
        )
    )

    return titers_chart

##### Also generate this chart for just a subset of individuals to show that there is heterogeneity (Supplemental Figure 2)

In [ ]:
#Titer plots for a subset of individuals to show diversity of responses
listofstrains =['A/Michigan/MISAPPHIREL365374393/2023_R142K','A/California/07/2009',
'A/Michigan/45/2015',
'A/Brisbane/02/2018',
'A/Hawaii/70/2019',
'A/Wisconsin/588/2019','A/Victoria/4897/2022_R142K','A/Wisconsin/67/2022']

titers2 = titers.loc[~titers['virus'].isin(listofstrains)]
titers2 = titers2.loc[titers2['serum'].str.contains("94_s016|79_s023|00_s017|51_s014|86_s004")]
condition_order =["pre-vaccination","post-vaccination"]
chart = run_titerchart_rot(titers, 3)
#titers_chart_html = "../results/aggregated_titers/titers_chart_selectedindividuals.html"
titers3 = titers2.sort_values(by='serum', ascending=False)

for df, chart_html, ncols in [
    (titers3, titers_chart_html, 1),
]:
    print("\n\n*********************************")
    chart = run_titerchart_rot(df, ncols)
    display(chart)
    print(f"Saving to {chart_html}")
    chart.save(chart_html)

#### Now create plots were we normalize the titer profiles for each individual to the median titer for that individual against a recent strain.
This allows us to look at which strains are less well neutralized for each individual, regardless of initial titer.

In [ ]:
#Calculate ratio of titer for each strain to median
titers_recentstrains = titers.loc[~titers['virus'].isin(non_recentstrainset)]
titers_recentstrains_median = titers_recentstrains.groupby(['serum','time']).median(numeric_only=True).reset_index().rename(columns={'titer':'median_titer'}).drop(columns=['titer_sem','n_replicates','YOB'])
titers_recentstrains_withsamplemedian = titers_recentstrains.merge(titers_recentstrains_median, on=['serum','time'])
titers_recentstrains_withsamplemedian['ratio_to_median'] = titers_recentstrains_withsamplemedian['titer'] /titers_recentstrains_withsamplemedian['median_titer']

titers_recentstrains_withsamplemedian_mediandev = titers_recentstrains_withsamplemedian.groupby(['virus','time','group']).quantile(0.1,numeric_only=True).reset_index().rename(columns={'titer':'median_titer', 'ratio_to_median':'min_ratio_to_median'}).drop(columns=['median_titer','titer_sem','YOB','n_replicates'])
titers_recentstrains_withsamplemedian_withmediandev = titers_recentstrains_withsamplemedian.merge(titers_recentstrains_withsamplemedian_mediandev, on=['virus','time','group'])

##### Generate plots for variation in titers pre-vaccination for both groups

In [ ]:
#Plot normalized titer ratios for both groups pre-vaccination
plot_range = [0.04,6]
range_ = ['mediumseagreen','mediumvioletred']
range_ = ['#BF3D6F','#55A079']

source = titers_recentstrains_withsamplemedian_withmediandev.loc[titers_recentstrains_withsamplemedian_withmediandev['group'].str.contains('Egg|Recombinant')]
source = source.loc[~source['virus'].isin(non_recentstrainset)]
source = source.loc[source['time'] =="pre-vaccination"]

NT50s_forselections_chart = alt.Chart(source).mark_line(point=False,strokeWidth=1, opacity=0.5).encode(
    y=alt.X("ratio_to_median",scale=alt.Scale(domain=plot_range, nice=False, clamp=True, type="log")),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",scale=alt.
                    Scale(range=range_)),
    detail= "participant",
#    facet=alt.Facet("group")
).properties(
    height=150,
    width = alt.Step(13))
meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=1).encode(
    y=alt.X("min_ratio_to_median",scale=alt.Scale(nice=False, type="log")),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale( range=range_)),
).properties(
    height=150,
    width = alt.Step(13))
meanline_chart_point = alt.Chart().mark_point(filled=True,shape="triangle-down").encode(
    y=alt.X("min_ratio_to_median",scale=alt.Scale( nice=False, type="log")),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale( range=range_)),
    size=alt.value(75),
    opacity=alt.value(1),
).properties(
    height=150,
    width = alt.Step(13))#
line_2foldless = alt.Chart().mark_rule().encode(y=alt.datum(0.5))
line_2foldmore = alt.Chart().mark_rule().encode(y=alt.datum(2))
line_atzero = alt.Chart().mark_rule(color="black", strokeDash=[1,0],strokeWidth=1.2).encode(y=alt.datum(1))
#line4 = alt.Chart().mark_rule().encode(y=alt.datum(0.25))

median_chart = alt.layer(line_atzero,NT50s_forselections_chart,line_2foldless,line_2foldmore,data = source).facet(row=alt.Row('group', sort=["Egg Vaccine Recipients", "Recombinant Vaccine Recipients"],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False).configure_rule(color="lightgrey",strokeDash=[2,2,2])

display(median_chart)
median_chart.save(Fig5_pre)

##### Generate plots for variation in titers post-vaccination for both groups

In [ ]:
#Plot normalized titer ratios for both groups post-vaccination
plot_range = [0.04,6]
range_ = ['mediumseagreen','mediumvioletred']
range_ = ['#BF3D6F','#55A079']

source = titers_recentstrains_withsamplemedian_withmediandev.loc[titers_recentstrains_withsamplemedian_withmediandev['group'].str.contains('Egg|Recombinant')]
source = source.loc[~source['virus'].isin(non_recentstrainset)]
source = source.loc[source['time'] =="post-vaccination"]

NT50s_forselections_chart = alt.Chart(source).mark_line(point=False,strokeWidth=1, opacity=0.5).encode(
    y=alt.X("ratio_to_median",scale=alt.Scale(domain=plot_range, nice=False, clamp=True, type="log")),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",scale=alt.
                    Scale(range=range_)),
    detail= "participant",
).properties(
    height=150,
    width = alt.Step(13))

meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=1).encode(
    y=alt.X("min_ratio_to_median",scale=alt.Scale(nice=False, type="log")),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale( range=range_)),
).properties(
    height=150,
    width = alt.Step(13))
meanline_chart_point = alt.Chart().mark_point(filled=True,shape="triangle-down").encode(
    y=alt.X("min_ratio_to_median",scale=alt.Scale( nice=False, type="log")),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale( range=range_)),
    size=alt.value(75),
    opacity=alt.value(1),
).properties(
    height=350,
    width = alt.Step(13))#
line_2foldless = alt.Chart().mark_rule().encode(y=alt.datum(0.5))
line_2foldmore = alt.Chart().mark_rule().encode(y=alt.datum(2))
line_atzero = alt.Chart().mark_rule(color="black", strokeDash=[1,0],strokeWidth=1.2).encode(y=alt.datum(1))

median_chart = alt.layer(line_atzero,NT50s_forselections_chart,line_2foldless,line_2foldmore,data = source).facet(row=alt.Row('group', sort=["Egg Vaccine Recipients", "Recombinant Vaccine Recipients"],header=alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=0))).configure_axis(grid=False).configure_rule(color="lightgrey",strokeDash=[2,2,2])

display(median_chart)
median_chart.save(Fig5_post)

In [ ]:
#Test if there are more individuals in egg-vaccinated group that have lower titers against clade 5a2a1 than in recombinant

#Define mannwhitneyu test to determine if an individual has lower titers against clade 5a2a1 viruses
def test_mannwhitneyu_inclade5a2a1(individual,notinclade5a2a1,inclade5a2a1):
    test = stats.mannwhitneyu(notinclade5a2a1,inclade5a2a1,alternative="greater")
    out = [individual,test[1]]
    return(out)

#Create dataframe with pvalues for each virus, using a Mann Whitney U test. Performed for egg and recombinant vaccine recipients for each individual.
source = titers_recentstrains_withsamplemedian_withmediandev
titers_totest = source.loc[source['time']=='post-vaccination']
sera = titers_totest['serum'].unique().tolist()

#Initialize a dataframe to hold pvalues
tests_and_pvalues = []
column_names = ["individual", "pvalue"]
for serum in sera:
    titers_totest_sera = titers_totest.loc[titers_totest['serum'] == serum]
    tests_and_pvalues.append(test_mannwhitneyu_inclade5a2a1(serum,titers_totest_sera.loc[titers_totest_sera['virus'].isin(non_5a2a1)]['titer'], titers_totest_sera.loc[titers_totest_sera['virus'].isin(clade_5a2a1)]['titer']))
allindividuals_withsigdiffin5a2a1clade = pd.DataFrame(tests_and_pvalues, columns=column_names)


#Add multiple testing correction
column_names = ["group","individual", "pvalue","pvalue_bh","reject_bh","pvalue_bonf","reject_bonf"]
#Initialize a dataframe with corrected pvalues
allindividuals_withsigdiffin5a2a1clade_correctedpvalue = []

def return_bh_and_bonf_corrected_pvalues(pvaluelist):
    #Apply Benjamini-Hochberg correction
    reject_bh, p_values_corrected_bh, _, _ = multipletests(pvaluelist, method='fdr_bh')
    #Apply Bonferroni correction (for comparison)
    reject_bonf, p_values_corrected_bonf, _, _ = multipletests(pvaluelist, method='bonferroni')
    return(reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf)

#Perform correction for all viruses
sera_pvalueslist = allindividuals_withsigdiffin5a2a1clade['pvalue'].tolist()
sera_names = allindividuals_withsigdiffin5a2a1clade['individual'].tolist()
reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf = return_bh_and_bonf_corrected_pvalues(sera_pvalueslist)
for i, name in enumerate(sera_names):
    if "PENN" in name:
        group ="Egg vaccine recipient"
    if "DRIVE" in name:
        group = "Recombinant vaccine recipient"
    out = [group,name,sera_pvalueslist[i],p_values_corrected_bh[i],reject_bh[i],p_values_corrected_bonf[i],reject_bonf[i]]
    allindividuals_withsigdiffin5a2a1clade_correctedpvalue.append(out)

#Create dataframe with corrected pvalues
allindividuals_withsigdiffin5a2a1clade_correctedpvalue_df = pd.DataFrame(allindividuals_withsigdiffin5a2a1clade_correctedpvalue, columns=column_names)

#Create table counting individuals with lower titers against 5a2a1 clade strains
allindividuals_withsigdiffin5a2a1clade_correctedpvalue_df['count'] = 1
numindividuals_withdiffin5a2a1 = allindividuals_withsigdiffin5a2a1clade_correctedpvalue_df.groupby(['group','reject_bonf']).sum(numeric_only=True).reset_index()
display(numindividuals_withdiffin5a2a1)

In [ ]:
#Perform Fishers exact test to determine if more individuals in egg-vaccinated group have lower titers against clade 5a2a1 strains than in recombinant
outcome_list = numindividuals_withdiffin5a2a1['count'].tolist()
table=[[outcome_list[0],outcome_list[1]],[outcome_list[2],outcome_list[3]]]
table
oddsratio, p = stats.fisher_exact(table, alternative='less')
print(oddsratio, p)

In [ ]:
#Test if there are more individuals in egg-vaccinated group that have lower titers against clade 5a2a1 than in recombinant pre-vaccination

#Define mannwhitneyu test to determine if an individual has lower titers against clade 5a2a1 viruses
def test_mannwhitneyu_inclade5a2a1(individual,notinclade5a2a1,inclade5a2a1):
    test = stats.mannwhitneyu(notinclade5a2a1,inclade5a2a1,alternative="greater")
    out = [individual,test[1]]
    return(out)

#Create dataframe with pvalues for each virus, using a Mann Whitney U test. Performed for egg and recombinant vaccine recipients for each individual.
source = titers_recentstrains_withsamplemedian_withmediandev
titers_totest = source.loc[source['time']=='pre-vaccination']
sera = titers_totest['serum'].unique().tolist()

#Initialize a dataframe to hold pvalues
tests_and_pvalues = []
column_names = ["individual", "pvalue"]
for serum in sera:
    titers_totest_sera = titers_totest.loc[titers_totest['serum'] == serum]
    tests_and_pvalues.append(test_mannwhitneyu_inclade5a2a1(serum,titers_totest_sera.loc[titers_totest_sera['virus'].isin(non_5a2a1)]['titer'], titers_totest_sera.loc[titers_totest_sera['virus'].isin(clade_5a2a1)]['titer']))
allindividuals_withsigdiffin5a2a1clade = pd.DataFrame(tests_and_pvalues, columns=column_names)


#Add multiple testing correction
column_names = ["group","individual", "pvalue","pvalue_bh","reject_bh","pvalue_bonf","reject_bonf"]
#Initialize a dataframe with corrected pvalues
allindividuals_withsigdiffin5a2a1clade_correctedpvalue = []

def return_bh_and_bonf_corrected_pvalues(pvaluelist):
    #Apply Benjamini-Hochberg correction
    reject_bh, p_values_corrected_bh, _, _ = multipletests(pvaluelist, method='fdr_bh')
    #Apply Bonferroni correction (for comparison)
    reject_bonf, p_values_corrected_bonf, _, _ = multipletests(pvaluelist, method='bonferroni')
    return(reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf)

#Perform correction for all viruses
sera_pvalueslist = allindividuals_withsigdiffin5a2a1clade['pvalue'].tolist()
sera_names = allindividuals_withsigdiffin5a2a1clade['individual'].tolist()
reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf = return_bh_and_bonf_corrected_pvalues(sera_pvalueslist)
for i, name in enumerate(sera_names):
    if "PENN" in name:
        group ="Egg vaccine recipient"
    if "DRIVE" in name:
        group = "Recombinant vaccine recipient"
    out = [group,name,sera_pvalueslist[i],p_values_corrected_bh[i],reject_bh[i],p_values_corrected_bonf[i],reject_bonf[i]]
    allindividuals_withsigdiffin5a2a1clade_correctedpvalue.append(out)

#Create dataframe with corrected pvalues
allindividuals_withsigdiffin5a2a1clade_correctedpvalue_df = pd.DataFrame(allindividuals_withsigdiffin5a2a1clade_correctedpvalue, columns=column_names)

#Create table counting individuals with lower titers against 5a2a1 clade strains
allindividuals_withsigdiffin5a2a1clade_correctedpvalue_df['count'] = 1
numindividuals_withdiffin5a2a1 = allindividuals_withsigdiffin5a2a1clade_correctedpvalue_df.groupby(['group','reject_bonf']).sum(numeric_only=True).reset_index()
display(numindividuals_withdiffin5a2a1)

In [ ]:
#Perform Fishers exact test to determine if more individuals in egg-vaccinated group have lower titers against clade 5a2a1 strains than in recombinant
outcome_list = numindividuals_withdiffin5a2a1['count'].tolist()
table=[[outcome_list[0],outcome_list[1]],[outcome_list[2],outcome_list[3]]]
table
oddsratio, p = stats.fisher_exact(table, alternative='less')
print(oddsratio, p)

In [ ]:
#Test if there are more individuals in egg-vaccinated group that have lower titers against clade 5a2a1 than in recombinant

#Define mannwhitneyu test to determine if an individual has lower titers against clade 5a2a1 viruses
def test_mannwhitneyu_incladeK142R(individual,K142_strains,R142_strains):
    test = stats.mannwhitneyu(K142_strains,R142_strains,alternative="greater")
    out = [individual,test[1]]
    return(out)

#Create dataframe with pvalues for each virus, using a Mann Whitney U test. Performed for egg and recombinant vaccine recipients for each individual.
source = titers_recentstrains_withsamplemedian_withmediandev
titers_totest = source.loc[source['time']=='post-vaccination']
sera = titers_totest['serum'].unique().tolist()

#Initialize a dataframe to hold pvalues
tests_and_pvalues = []
column_names = ["individual", "pvalue"]
for serum in sera:
    titers_totest_sera = titers_totest.loc[titers_totest['serum'] == serum]
    tests_and_pvalues.append(test_mannwhitneyu_incladeK142R(serum,titers_totest_sera.loc[titers_totest_sera['virus'].isin(K142_strains)]['titer'], titers_totest_sera.loc[titers_totest_sera['virus'].isin(R142_strains)]['titer']))
allindividuals_withsigdiffinK142R = pd.DataFrame(tests_and_pvalues, columns=column_names)


#Add multiple testing correction
column_names = ["group","individual", "pvalue","pvalue_bh","reject_bh","pvalue_bonf","reject_bonf"]
#Initialize a dataframe with corrected pvalues
allindividuals_withsigdiffinK142R_correctedpvalue = []

def return_bh_and_bonf_corrected_pvalues(pvaluelist):
    #Apply Benjamini-Hochberg correction
    reject_bh, p_values_corrected_bh, _, _ = multipletests(pvaluelist, method='fdr_bh')
    #Apply Bonferroni correction (for comparison)
    reject_bonf, p_values_corrected_bonf, _, _ = multipletests(pvaluelist, method='bonferroni')
    return(reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf)

#Perform correction for all viruses
sera_pvalueslist = allindividuals_withsigdiffinK142R['pvalue'].tolist()
sera_names = allindividuals_withsigdiffinK142R['individual'].tolist()
reject_bh,p_values_corrected_bh,reject_bonf,p_values_corrected_bonf = return_bh_and_bonf_corrected_pvalues(sera_pvalueslist)
for i, name in enumerate(sera_names):
    if "PENN" in name:
        group ="Egg vaccine recipient"
    if "DRIVE" in name:
        group = "Recombinant vaccine recipient"
    out = [group,name,sera_pvalueslist[i],p_values_corrected_bh[i],reject_bh[i],p_values_corrected_bonf[i],reject_bonf[i]]
    allindividuals_withsigdiffinK142R_correctedpvalue.append(out)

#Create dataframe with corrected pvalues
allindividuals_withsigdiffinK142R_correctedpvalue = pd.DataFrame(allindividuals_withsigdiffinK142R_correctedpvalue, columns=column_names)

#Create table counting individuals with lower titers against 5a2a1 clade strains
allindividuals_withsigdiffinK142R_correctedpvalue['count'] = 1
numindividuals_withdiffinK142R = allindividuals_withsigdiffinK142R_correctedpvalue.groupby(['group','reject_bonf']).sum(numeric_only=True).reset_index()
display(numindividuals_withdiffinK142R)

In [ ]:
#Perform Fishers exact test to determine if more individuals in egg-vaccinated group have lower titers against clade K142R strains than in recombinant
outcome_list = numindividuals_withdiffinK142R['count'].tolist()
table=[[outcome_list[0],outcome_list[1]],[outcome_list[2],outcome_list[3]]]
table
oddsratio, p = stats.fisher_exact(table, alternative='less')
print(oddsratio, p)

#### These plots are solely for visualization of data, and are not included in the manuscript

##### Generate an html with pre- and post- vaccination charts for all individuals in both groups

In [ ]:
#Generate HTML titer plot with pre- and post- vaccination titers for all individuals in both groups
listofstrains =['A/California/07/2009','A/Michigan/45/2015','A/Brisbane/02/2018','A/Hawaii/70/2019','A/Wisconsin/588/2019','A/Wisconsin/67/2022','A/Michigan/MISAPPHIREL365374393/2023','A/Oregon/Flu-OHSU-241140095/2023','A/Victoria/4897/2022_IVR238']

titers_bothgroups = titers

titers_bothgroups = titers_bothgroups.loc[titers_bothgroups['group'].str.contains("Egg|Recombinant")]


condition_order =["pre-vaccination","post-vaccination"]
chart = run_titerchart_rot(titers, 1)
#titers_chart_html = snakemake.output.titers_html

for df, chart_html, ncols in [
    (titers_bothgroups, titers_chart_html, 1),
]:
    print("\n\n*********************************")
    chart = run_titerchart_rot(df, ncols)
#    display(chart)
    print(f"Saving to {chart_html}")
    chart.save(chart_html)

##### Generate heatmaps to look at post-vaccination titers for all individuals in both groups simultaneously

In [ ]:
#Plot heatmap for each individuals titers post vaccination. Make the titers that are at the upper-bound obvious, cell-vaccine
postvaxtiter_timesorted = titers.loc[titers['time'] == 'post-vaccination'].sort_values(by=['YOB'], ascending=True)
postvaxtiter_timesorted = postvaxtiter_timesorted.loc[~postvaxtiter_timesorted['titer_bound'].str.contains('upper')]
postvaxtiter_timesorted = postvaxtiter_timesorted.loc[~postvaxtiter_timesorted['strain_category'].str.contains("Vaccine")]
postvaxtiter_timesorted_recombinant = postvaxtiter_timesorted.loc[postvaxtiter_timesorted['group'].str.contains("Recombinant")]
postvaxtiter_timesorted_egg = postvaxtiter_timesorted.loc[postvaxtiter_timesorted['group'].str.contains("Egg")]
sorted_age = postvaxtiter_timesorted['participant'].unique().tolist()


heatmap_recombinant = alt.Chart(postvaxtiter_timesorted_recombinant).mark_rect().encode(
    y=alt.X("participant", sort=sorted_age,
            title="participant"),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("titer",title="titer",scale=alt.
                    Scale(type='log',domain=[40,4000]))
).properties(
    height=350,
    width = 450,title="Recombinant Vaccine Recipients")
display(heatmap_recombinant)

heatmap_egg = alt.Chart(postvaxtiter_timesorted_egg).mark_rect().encode(
    y=alt.X("participant", sort=sorted_age,
            title="participant"),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("titer",title="titer",scale=alt.
                    Scale(type='log',domain=[40,4000]))
).properties(
    height=350,
    width = 450,title="Egg Vaccine Recipients")
display(heatmap_egg)